In [2]:
# INTRODUCTION

#The objective of the study is to analyse the flight booking dataset obtained from “Ease My Trip” website and to conduct 
#various statistical hypothesis tests in order to get meaningful information from it. 
#The 'Linear Regression' statistical algorithm would be used to train the dataset and predict a continuous target 
#variable. 'Easemytrip' is an internet platform for booking flight tickets, and hence a platform that potential 
#passengers use to buy tickets. A thorough study of the data will aid in the discovery of valuable insights that 
#will be of enormous value to passengers.

# https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction?resource=download

#-------------------
#Research Questions
#-------------------
#The aim of our study is to answer the below research questions:
#a) Does price vary with Airlines?
#b) How is the price affected when tickets are bought in just 1 or 2 days before departure?
#c) Does ticket price change based on the departure time and arrival time?
#d) How the price changes with change in Source and Destination?
#e) How does the ticket price vary between Economy and Business class?

#DATA COLLECTION AND METHODOLOGY
#Octoparse scraping tool was used to extract data from the website. Data was collected in two parts: one for 
#economy class tickets and another for business class tickets. A total of 300261 distinct flight booking options was 
#extracted from the site. Data was collected for 50 days, from February 11th to March 31st, 2022.
#Data source was secondary data and was collected from Ease my trip website.

#DATASET
#Dataset contains information about flight booking options from the website Easemytrip for flight travel between 
#India's top 6 metro cities. There are 300261 datapoints and 11 features in the cleaned dataset.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error,mean_absolute_percentage_error
from IPython.display import display 
import json,pickle

from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, LSTM
from keras import callbacks

from sklearn.model_selection import GridSearchCV, cross_val_score, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor

from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, pearsonr, spearmanr, kendalltau, \
    f_oneway, kruskal

from sklearn.model_selection import RepeatedKFold, KFold
from skopt import BayesSearchCV, space

2023-05-23 16:14:32.588358: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 16:14:32.694207: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-23 16:14:32.694239: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-23 16:14:32.724952: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-23 16:14:33.260005: W tensorflow/stream_executor/platform/de

In [4]:
flights = pd.read_csv("Clean_Dataset.csv")

In [5]:
flights

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955
...,...,...,...,...,...,...,...,...,...,...,...,...
300148,300148,Vistara,UK-822,Chennai,Morning,one,Evening,Hyderabad,Business,10.08,49,69265
300149,300149,Vistara,UK-826,Chennai,Afternoon,one,Night,Hyderabad,Business,10.42,49,77105
300150,300150,Vistara,UK-832,Chennai,Early_Morning,one,Night,Hyderabad,Business,13.83,49,79099
300151,300151,Vistara,UK-828,Chennai,Early_Morning,one,Evening,Hyderabad,Business,10.00,49,81585


In [6]:
flights = flights.drop(columns=['Unnamed: 0'], axis=1)

In [7]:
encoder = LabelEncoder()

for col in flights.columns[:-3]:
    flights[col] = encoder.fit_transform(flights[col])

In [8]:
flights

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,4,1408,2,2,2,5,5,1,2.17,1,5953
1,4,1387,2,1,2,4,5,1,2.33,1,5953
2,0,1213,2,1,2,1,5,1,2.17,1,5956
3,5,1559,2,4,2,0,5,1,2.25,1,5955
4,5,1549,2,4,2,4,5,1,2.33,1,5955
...,...,...,...,...,...,...,...,...,...,...,...
300148,5,1477,1,4,0,2,3,0,10.08,49,69265
300149,5,1481,1,0,0,5,3,0,10.42,49,77105
300150,5,1486,1,1,0,5,3,0,13.83,49,79099
300151,5,1483,1,1,0,2,3,0,10.00,49,81585


In [9]:
X = flights[flights.columns[:-1]]
y = flights[flights.columns[-1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [10]:
from xgboost import XGBRegressor

In [ ]:
regr = XGBRegressor()

params = dict()
params['max_depth'] = space.Integer(0, 30)
params['n_estimators'] = space.Integer(100, 1000)
params['learning_rate'] = space.Real(0.01, 1.0, prior='log-uniform')
params['gamma'] = space.Real(1e-9, 0.5, prior='log-uniform')
params['subsample'] = space.Real(0.1, 1.0)

cv = KFold(n_splits=5, shuffle=True)
search = BayesSearchCV(estimator=regr, search_spaces=params, n_jobs=-1, cv=cv, scoring='r2', n_iter=20)
search.fit(X_train, y_train)

In [20]:
search.best_params_

OrderedDict([('gamma', 5.83500047748077e-09),
             ('learning_rate', 0.08100337275722415),
             ('max_depth', 21),
             ('n_estimators', 781),
             ('subsample', 0.9372890793195705)])

In [21]:
regr = search.best_estimator_

In [22]:
y_hat = regr.predict(X_test)

mae = mean_absolute_error(y_test, y_hat)
mape = np.mean(np.abs((y_test - y_hat) / y_test)) * 100
mse = mean_squared_error(y_test, y_hat)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_hat)

In [23]:
print('mae', mae)
print('mape', mape)
print('mse', mse)
print('rme', rmse)
print('r2', r2)

mae 835.7763371430416
mape 5.903665453891699
mse 4686852.132787442
rme 2164.913885767155
r2 0.9909218613495339


In [24]:
for item in zip(y_test, y_hat):
    print(item)

(79588, 58172.07)
(13411, 11396.946)
(6395, 6394.738)
(6489, 8196.387)
(5177, 5207.923)
(6985, 6961.6987)
(4792, 4816.3105)
(50969, 51319.88)
(8112, 8685.267)
(71058, 73451.72)
(4933, 4885.0654)
(4500, 4434.7676)
(5817, 5739.0464)
(8024, 8461.747)
(8733, 6958.968)
(4266, 2217.824)
(10324, 10273.824)
(78898, 76349.94)
(4758, 4718.3105)
(6308, 6953.4263)
(58315, 60428.527)
(9578, 7811.477)
(2675, 2468.457)
(7494, 7456.4673)
(2974, 6401.38)
(22140, 22140.67)
(11383, 14414.224)
(5810, 5898.9272)
(4556, 4696.4106)
(4469, 4236.222)
(5632, 5791.7803)
(3096, 3095.7805)
(45693, 45672.47)
(42546, 42594.42)
(69244, 69575.08)
(2750, 3779.542)
(4020, 3956.01)
(4969, 5089.0645)
(4971, 5046.56)
(6924, 7181.849)
(51817, 51217.19)
(61404, 61692.336)
(80544, 66976.71)
(6032, 6039.297)
(15889, 13695.633)
(5817, 5895.57)
(12927, 12888.236)
(3291, 3440.0864)
(4672, 4527.6733)
(52063, 52128.21)
(13054, 12727.437)
(3000, 3009.8)
(4634, 4076.2388)
(53743, 53854.96)
(20000, 20075.01)
(2603, 2793.952)
(7352, 74

In [30]:
importance = regr.get_booster().get_score(importance_type='gain')
pd.DataFrame(importance, index=[0]).T.sort_values(by=0)

,0
airline,9.862682e+05
days_left,1.341506e+06
arrival_time,1.460926e+06
departure_time,1.475252e+06
destination_city,2.873798e+06
flight,2.992350e+06
stops,3.069503e+06
source_city,4.557248e+06
duration,7.094332e+06
class,5.067220e+08
